Ingest Data Sources & Create Views

In [0]:
# Path
volume_path = "/Volumes/healthcareclaims/default/data"

# 1. Ingest data from CSV and JSON into DataFrame
claims_df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load(f"{volume_path}/claims_batch.csv")
members_df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load(f"{volume_path}/members.csv")
providers_df = spark.read.format("json").option("multiline", "true").load(f"{volume_path}/providers.json")
diagnosis_df = spark.read.format("csv").option("header", "true").option("inferSchema", "true").load(f"{volume_path}/diagnosis_ref.csv")

# Create temporary Views from DataFrames
claims_df.createOrReplaceTempView("raw_claims_vw")
members_df.createOrReplaceTempView("raw_members_vw")
providers_df.createOrReplaceTempView("raw_providers_vw")
diagnosis_df.createOrReplaceTempView("raw_diagnosis_vw")

 Data Profiling & Cleansing

In [0]:
deduplicated_claims_sql = """
WITH NumberedClaims AS (
  SELECT
    *,
    ROW_NUMBER() OVER(PARTITION BY ClaimID, MemberID, ProviderID ORDER BY IngestTimestamp DESC) as rn
  FROM raw_claims_vw
)
SELECT * FROM NumberedClaims WHERE rn = 1
"""
deduplicated_claims_df = spark.sql(deduplicated_claims_sql)
deduplicated_claims_df.createOrReplaceTempView("deduplicated_claims_vw")

# filter out: null
spark.sql("""
  SELECT *
  FROM deduplicated_claims_vw
  WHERE ClaimID IS NOT NULL AND MemberID IS NOT NULL AND ProviderID IS NOT NULL
""").createOrReplaceTempView("cleansed_claims_vw") 

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW cleansed_typed_claims_vw AS
SELECT
  ClaimID,
  MemberID,
  ProviderID,
  CAST(ClaimDate AS DATE) AS ClaimDate,
  CAST(ServiceDate AS DATE) AS ServiceDate,
  CAST(Amount AS DOUBLE) AS Amount,
  Status,
  ICD10Codes,
  CPTCodes,
  ClaimType
FROM cleansed_claims_vw;

SELECT * FROM cleansed_typed_claims_vw LIMIT 5;

Key Validation

In [0]:
%sql
-- check duplicate
SELECT
  ClaimID, MemberID, ProviderID,
  COUNT(*) as occurrences
FROM cleansed_typed_claims_vw
GROUP BY ClaimID, MemberID, ProviderID
HAVING COUNT(*) > 1;

-- expected returned rows = No rows returned

In [0]:
%sql 
SELECT c.ClaimID, c.MemberID
FROM cleansed_typed_claims_vw c
LEFT ANTI JOIN raw_members_vw m ON c.MemberID = m.MemberID;

-- expected returned rows = No rows returned

Parse Nested JSON & Enrich Data

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW providers_exploded_vw AS
SELECT
  ProviderID,
  Name,
  IsActive,
  TIN,
  explode(Specialties) as Specialty,
  explode(Locations) as Location
FROM raw_providers_vw;

-- Join ตารางทั้งหมดเข้าด้วยกันเพื่อสร้าง Enriched View
CREATE OR REPLACE TEMP VIEW enriched_claims_vw AS
SELECT
  c.ClaimID,
  c.ClaimDate,
  c.Amount,
  c.Status,
  c.ClaimType,
  m.Name AS MemberName,
  m.Region AS MemberRegion,
  m.PlanType,
  p.Name AS ProviderName,
  p.Specialty AS ProviderSpecialty,
  p.Location.City AS ProviderCity,
  p.Location.State AS ProviderState
FROM cleansed_typed_claims_vw c
JOIN raw_members_vw m ON c.MemberID = m.MemberID
JOIN providers_exploded_vw p ON c.ProviderID = p.ProviderID;

SELECT * FROM enriched_claims_vw LIMIT 10;

Pivot Data for Reporting

In [0]:
# Read data from Enriched View
enriched_df = spark.table("enriched_claims_vw")

# Pivot data for the report
pivot_df = (enriched_df
            .groupBy("MemberRegion")
            .pivot("ClaimType", ["Inpatient", "Outpatient", "Pharmacy", "Dental"]) # pivot columns
            .sum("Amount")  
            .na.fill(0)) # fill 0 if null or empty

pivot_df.show()

Fraud Scoring with CASE/WHEN and UDF

In [0]:
%sql
-- CASE/WHEN
CREATE OR REPLACE TEMP VIEW claims_with_fraud_score_vw AS
SELECT
  *,
  -- Logic
  CASE
    WHEN Amount > 4000 THEN 10  
    ELSE 0
  END +
  CASE
    WHEN DAYOFWEEK(ClaimDate) IN (1, 7) THEN 5  
    ELSE 0
  END +
  CASE
    WHEN Status = 'Rejected' THEN 2
    ELSE 0
  END AS fraud_score
FROM enriched_claims_vw;

SELECT ClaimID, Amount, DAYOFWEEK(ClaimDate) as DoW, Status, fraud_score
FROM claims_with_fraud_score_vw
ORDER BY fraud_score DESC
LIMIT 10;

In [0]:
# ===== UDF =====

from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType

# Logic
def calculate_complexity_score(claim_type, specialty):
  score = 0
  if claim_type == "Inpatient" and specialty == "Surgery":
    score = 15
  elif claim_type == "Pharmacy":
    score = 2
  else:
    score = 5
  return score

# function to UDF
complexity_udf = udf(calculate_complexity_score, IntegerType())

# UDF as DataFrame
final_df = spark.table("claims_with_fraud_score_vw")
final_df_with_udf = final_df.withColumn("complexity_score", complexity_udf(final_df.ClaimType, final_df.ProviderSpecialty))

final_df_with_udf.select("ClaimID", "ClaimType", "ProviderSpecialty", "fraud_score", "complexity_score").show(10)